# `Installs`

In [1]:
!pip install crewai==0.51.0 crewai_tools==0.8.3 langchain_community==0.2.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements.

In [2]:
#!pip install CrewAI crewai_tools langchain_community


In [3]:
#!pip install pandas openpyxl


In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [5]:
import json
from crewai import Agent, Task, Crew, Process

In [6]:
import os


from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')
serper_api_key = userdata.get('SERPER_API_KEY')


os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')



In [7]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Creating tools

In [55]:
# # Creating tools
!pip install requests  # Install the requests library
import requests

In [56]:
#pip install --upgrade crewai_tools


In [57]:
from crewai_tools import BaseTool
class CustomSerperDevTool(BaseTool):
    name: str = "Custom Serper Dev Tool"
    description: str = "Search the internet for Patents research."

    # @record_tool(tool_name="Custom Serper Dev Tool")
    def _run(self, query: str) -> str:
        """
        Search the internet for Patents.
        """

        url = "https://google.serper.dev/patents"

        payload = json.dumps({
            "q": query,
            "num": 20,
            "autocorrect": False,
            "tbs": "qdr:d"
        })

        headers = {
            'X-API-KEY': os.getenv('SERPER_API_KEY'),
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            response_data = response.json()

        # Convert the news data back to a JSON string
        return json.dumps(response_data, indent=2)


In [58]:

# Custom file writer function
def write_to_file(content, directory='project', filename='output.txt'):
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_path = os.path.join(directory, filename)

    # Ensure content is a string before writing
    if not isinstance(content, str):
        content = str(content)

    with open(file_path, 'w') as file:
        file.write(content)

    print(f"Content written to {file_path}")

In [59]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool, \
                         DallETool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool, \
                         VisionTool



In [60]:
#from google.colab import drive
#drive.mount('/content/drive')

In [61]:
#Access Google Drive to write files
#from google.colab import drive
#drive.mount('/content/drive')

In [62]:
#read_inputs = DirectoryReadTool(directory='/content/drive/MyDrive/Creative_AI/research_engine/Inputs')
#read_outputs = DirectoryReadTool(directory='/content/drive/MyDrive/Creative_AI/research_engine/Outputs')
#read_baseline= FileReadTool(file_path='/content/drive/MyDrive/Creative_AI/research_engine/Outputs/knowledge_baseline.txt')
#file_read_tool = FileReadTool()
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
website_search_tool = WebsiteSearchTool()
#vision_tool = VisionTool(model="dall-e-3",
#                       size="1024x1024",
#                       quality="standard",
#                       n=1)


In [63]:
# Direct file reading using Python standard library
#file_path = '/content/drive/MyDrive/Creative_AI/research_engine/Inputs/test.txt'
#with open(file_path, 'r') as file:
#    content = file.read()

#Check if content is being read
#print(content)


# AGENTS

In [64]:
#Not use here

manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the research team and ensure the production of world-class research reports",
    backstory=(
        "You are a very highly experienced research project manager, and you make sure work is always completed with extremely high standard."
        "You make sure to include multiple revision loops to check the quality and truthfulness of information."
        "Make sure there is first a stage of knowledge collection, then analysis and interpretation, before the report is finalized."
        "Ensure the content is complete, truthful, relevant to the {topic}, {purpose} and {context}."
        "If anything is missing or not at the right level of quality, send it back for revision.\n"
        "Research topic: {topic}\n"
        "Research purpose: {purpose}\n"
        "Research context: {context}\n"

    ),
    allow_delegation=True,
    verbose=True,
    memory=True,
    max_iter=2
)

In [65]:
strategist_agent = Agent(
    role="Strategist",
    goal="Unpack the challenge provided",
    backstory=(
        "As an expert strategist, "
        "you are skilled at dissecting and unpacking challenges into its constituent elements. "
    ),
    allow_delegation=True,
    verbose=True,
    memory=True,
    max_iter=2
)

In [66]:
researcher_agent = Agent(
    role="Researcher",
    goal="Find relevant information to answer questions you are asked",
    backstory=(
        "As a diligent and methodical researcher, "
        "you are tasked with gathering accessible and relevant information"
        "with the tools provided. "
        "You excel at sifting through vast amounts of information "
        "to distill the most relevant points. "

        "Your keen eye for detail and persistence ensure that "
        "no valuable information is missed, setting the stage for in-depth analysis."

    ),
    allow_delegation=True,
    verbose=True,
    memory=True,
    max_iter=2
)

In [67]:
scraper_agent = Agent(
    role="Scraper",
    goal="Extract detailed information from patent documents",
    backstory=(
        "As an expert scraper, "
        "you excel at retrieving and structuring relevant data from patent documents, ensuring that abstracts, claims, "
        "and other technical information are accurately captured and organized for further analysis."
    ),
    allow_delegation=True,
    verbose=True,
    memory=True,
    max_iter=2
)

In [68]:
writer_agent = Agent(
    role="Writer",
    goal="Summarize technical information from patents into clear, digestible formats",
    backstory=(
        "As a skilled technical writer, "
        "you specialize in turning complex technical information into concise and clear summaries that highlight key innovations, "
        "claims, and potential applications, ensuring they are easily understood by both technical and non-technical audiences."
    ),
    allow_delegation=True,
    verbose=True,
    memory=True,
    max_iter=2
)

# Tasks

In [69]:
generate_queries_task = Task(
    description=(
        "Based on the provided topic (e.g., 'biodegradable packaging for food'), generate a set of related queries "
        "to run (e.g., 'biodegradable food packaging', 'compostable packaging materials', etc.). These queries should be semantically related "
        "and cover possible variations of the original query."

        "Topic: {topic}\n"
    ),
    expected_output=(
        "A list queries related to {topic} that can be used for further searching. "
        "Example: ['biodegradable food packaging', 'compostable packaging materials', 'eco-friendly packaging for food', 'plant-based packaging for perishables']"
    ),
    tools=[],
    agent=strategist_agent
)

In [70]:
patent_search_task = Task(
    description=(
        "For each of the queries generated in Task 1, use the search tool to query Google Patents and find "
        "the top 5 relevant search results for each query. Ensure the patents selected are closely related to the original topic."
    ),
    expected_output=(
        "A list of the top 5 patent results for each query, including their titles, URLs, and brief descriptions. "

    ),
    tools=[CustomSerperDevTool(),scrape_tool],
    agent=researcher_agent
)

In [71]:
scrape_content_task = Task(
    description=(
        "For each of the top 5 patent results obtained in Task 2, scrape the detailed content from the patent pages. "
        "Ensure that the abstract, claims, and any relevant technical information are captured for each patent."
    ),
    expected_output=(
        "A collection of the scraped content for each patent, including abstracts, claims, and any important technical details. "

    ),
    tools=[scrape_tool],
    agent=scraper_agent
)


In [72]:
summarize_task = Task(
    description=(
        "Using the scraped content from Task 3, summarize each patent in a concise format. "
        "The summary should focus on the key claims, innovations, and potential applications of each patent. "
        "Ensure that technical details are translated into clear, digestible information for easy review."
    ),
    expected_output=(
        "A summary of each patent that highlights the main claims, innovations, and applications. "
        "Example: 'Patent 1: This biodegradable packaging material is designed for perishable foods and is compostable, offering an eco-friendly alternative to plastic packaging. The innovation lies in the use of plant-based fibers and the potential for mass adoption in food supply chains.'"
    ),
    tools=[],  # Placeholder tool for summarizing patent content
    agent=writer_agent
)


# Creating crews

In [73]:
#Play with planning, process, manager

crew = Crew(
    agents=[strategist_agent, researcher_agent, scraper_agent, writer_agent],
    tasks=[
        generate_queries_task,
        patent_search_task,
        scrape_content_task,
        summarize_task,
    ],

    #process=Process.hierarchical,
    #manager_agent=manager,
    #manager_llm=manager_llm,

    process=Process.sequential,
    #planning=True,
    verbose=True,
	  memory=True,
    #cache=False,
    #share_crew=False,
    #output_log_file="outputs/content_plan_log.txt",
    #max_rpm=50,
    output_name='summaries'

)

# Run step by step

In [74]:
# Inputs with the correct title and topic
inputs = {
    "topic": "Coffee bean coating",
    "title": "Coffee Bean Coating Technologies",
    "context": "We are a consulting firm advising our clients on innovative technologies",
    "purpose": "Writing a future-oriented report for a panel of global corporate, IGO, and NGO stakeholders",
    "sector": "Energy providers"
}

This cell will take time for the conclusion, wants results fater then set parameter of max_iter= 3

In [75]:
# Run crew

summaries= crew.kickoff(inputs=inputs)


# Save the report to a file
#write_to_file(summaries, directory='/content/drive/MyDrive/Creative_AI/research_engine/Outputs', filename='playground.txt')

 [2024-10-21 04:13:58][DEBUG]: == Working Agent: Strategist
 [2024-10-21 04:13:58][INFO]: == Starting Task: Based on the provided topic (e.g., 'biodegradable packaging for food'), generate a set of related queries to run (e.g., 'biodegradable food packaging', 'compostable packaging materials', etc.). These queries should be semantically related and cover possible variations of the original query.Topic: Coffee bean coating



> Entering new CrewAgentExecutor chain...
I need to gather specific queries related to the topic of coffee bean coating, ensuring they cover various aspects such as applications, benefits, and environmental impact, while also considering popular search terms.

Action: Delegate work to coworker  
Action Input: {"task":"Generate a set of queries related to coffee bean coating, focusing on specific applications, benefits, and environmental impacts. Include terms like 'edible coatings', 'flavor enhancement', and 'antimicrobial properties'. Consider categorizing the que

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  3.19it/s]


I need to gather relevant patents for each of the queries provided regarding coffee bean coatings. I will use the search tool to find the top 5 patent results for each query. 

Action: Custom Serper Dev Tool  
Action Input: {"query": "effective edible coatings for enhancing coffee flavor"}  

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.97it/s]


 

{
  "searchParameters": {
    "q": "effective edible coatings for enhancing coffee flavor",
    "type": "patents",
    "num": 20,
    "autocorrect": false,
    "tbs": "qdr:d",
    "engine": "google"
  },
  "organic": [
    {
      "title": "A kind of spicy serious flavoring food and preparation method thereof",
      "snippet": "The invention discloses a kind of spicy serious flavoring food, processing method includes the following steps\uff1aA. infusion slaughterhouse\uff1bB. frying bottom material\uff1bC. the red oil of spicy old fire pot is refined\uff1bD. assembly packaging.Spicy serious flavoring food produced by the invention maintains \u2026",
      "link": "https://patents.google.com/patent/CN108077876A/en",
      "priorityDate": "2018-01-26",
      "filingDate": "2018-01-26",
      "publicationDate": "2018-05-29",
      "inventor": "\u949f\u6587",
      "assignee": "\u91cd\u5e86\u5e02\u5f00\u5dde\u533a\u5341\u91cc\u7af9\u6eaa\u519c\u4e1a\u5f00\u53d1\u6709\u9650\u516c\u53f8"

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  4.12it/s]


Thought: I need to gather more relevant patents for the first query regarding effective edible coatings for enhancing coffee flavor. I will continue to the next query. 

Action: Custom Serper Dev Tool  
Action Input: {"query": "antimicrobial properties in coffee bean coatings shelf life"}   

{
  "searchParameters": {
    "q": "antimicrobial properties in coffee bean coatings shelf life",
    "type": "patents",
    "num": 20,
    "autocorrect": false,
    "tbs": "qdr:d",
    "engine": "google"
  },
  "organic": [
    {
      "title": "Buffered microencapsulated compositions and methods",
      "snippet": "A microcapsule composition comprising at least one polymer substantially disposed as a semi-permeable shell around an aqueous buffered solution and at least one agent, wherein the agent permeates the shell, and wherein the composition is suitable for delivery to a mammal. This invention also \u2026",
      "link": "https://patents.google.com/patent/US10688026B2/en",
      "priorityDat

In [76]:
summaries

CrewOutput(raw='1. **Patent 1: Natural Edible Coating with Flavor Enhancement**  \n   - **Main Claims**: This patent claims a natural, edible coating that not only protects coffee beans but also enhances their flavor. The coating is made from food-grade materials, ensuring safety for consumption.  \n   - **Innovations**: The innovative aspect of this coating lies in its dual function; it acts as a barrier against environmental factors while infusing the beans with additional flavors during storage. It uses natural ingredients that are typically safe and non-toxic.  \n   - **Potential Applications**: This coating could be used by coffee producers and roasters to improve the taste of their coffee products, making them more appealing to consumers. It can also be beneficial for extending the shelf life of coffee beans by preventing moisture loss and flavor degradation.  \n\n2. **Patent 2: Flavor Infusion Coating for Coffee Beans**  \n   - **Main Claims**: This patent describes a specific c

In [77]:
print(summaries)

1. **Patent 1: Natural Edible Coating with Flavor Enhancement**  
   - **Main Claims**: This patent claims a natural, edible coating that not only protects coffee beans but also enhances their flavor. The coating is made from food-grade materials, ensuring safety for consumption.  
   - **Innovations**: The innovative aspect of this coating lies in its dual function; it acts as a barrier against environmental factors while infusing the beans with additional flavors during storage. It uses natural ingredients that are typically safe and non-toxic.  
   - **Potential Applications**: This coating could be used by coffee producers and roasters to improve the taste of their coffee products, making them more appealing to consumers. It can also be beneficial for extending the shelf life of coffee beans by preventing moisture loss and flavor degradation.  

2. **Patent 2: Flavor Infusion Coating for Coffee Beans**  
   - **Main Claims**: This patent describes a specific coating that encapsulat

In [78]:
print(type(summaries))

<class 'crewai.crews.crew_output.CrewOutput'>
